## 4. 랜덤포레스트 분류 모델
- 여러 개의 의사결정트리의 결과값을 평균낸 것으로 결과를 사용하는 분류 모델

### 라이브러리 불러오기

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import CountVectorizer

### 데이타 불러오기
- 전처리된 텍스트 파일 불러오기

In [1]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'
TEST_SIZE = 0.2
RANDOM_SEED = 42

In [4]:
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

In [5]:
reviews = list(train_data['review'])
y = np.array(train_data['sentiment'])

### CountVectorizer를 활용한 벡터화
- 사이킷런의 CountVectorizer를 이용하여 모델의 입력값을 생성
- CountVectorizer 객체를 불러온 후 리뷰 텍스트 데이타에 적용시켜 벡터화된 값을 변수에 할당
- 분석 단위를 하나의 단어로 지정: analyzer = "word", 각 벡터의 최대 길이 5000으로 지정

In [6]:
vectorizer = CountVectorizer(analyzer = "word", max_features = 5000) 

train_data_features = vectorizer.fit_transform(reviews)

In [7]:
train_data_features

<25000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1975048 stored elements in Compressed Sparse Row format>

- (25000, 5000)크기의 행렬로 구성
- 25000개의 리뷰 데이타가 각각 5000개의 특징값을 갖는 벡터로 표현되어 있음을 의미

In [9]:
type(train_data_features)

scipy.sparse.csr.csr_matrix

### 학습 데이타와 테스트로 분리

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_data_features, y, \
                                                    test_size=TEST_SIZE, random_state=RANDOM_SEED)

### 랜덤포레스트 모델 생성

In [11]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 분류기에  100개 의사 결정 트리를 사용한다.
forest = RandomForestClassifier(n_estimators = 100) 

### 모델 학습

In [12]:
# 단어 묶음을 벡터화한 데이터와 정답 데이터를 가지고 학습
forest.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### 모델 예측과 평가

In [35]:
predict = forest.predict(X_test)
predict

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [36]:
y_test

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [38]:
# 정확도
print("Accuracy: %f" % forest.score(X_test, y_test))  

Accuracy: 0.843000


### 평가 데이터셋으로 성능 평가

#### 평가 데이타 불러오기
- 평가 데이타도 각 리뷰를 단어들의 리스트로 만든다.
- id와 결과값을 csv에 담기 위해 id 분리

In [42]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_reviews = list(test_data['review'])
ids = list(test_data['id'])

- 모델을 적용하기 위해 학습 데이타에 적용한 것처럼 벡터화
- 학습 데이타에 적용시킨 vectorizer를 사용

In [43]:
test_data_features = vectorizer.transform(test_reviews)

### 데이터 제출하기
- 벡터화된 평가 데이타를 학습시킨 랜덤포레스트 모델에 적용하고
- id와 예측값을 판다스 데이타프레임 형식으로 만든 후 
- csv 파일로 저장한다.
- 캐글에 csv 파일 제출한 후 정확도 확인

In [44]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
# 위에서 만든 랜덤 포레스트 분류기를 통해 예측값을 가져온다.
result = forest.predict(test_data_features)

# 판다스 데이터 프레임을 통해 데이터를 구성해서 output에 넣는다.
output = pd.DataFrame( data={"id": ids, "sentiment": result} )

# csv파일로 만든다.
output.to_csv( DATA_OUT_PATH + "Bag_of_Words_model.csv", index=False, quoting=3 )